In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tqdm.auto import tqdm

import sys
sys.path.append("../../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-08-30 22:16:29 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-08-30 22:16:29 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-08-30 22:16:29 __main__ INFO     transformers.__version__='4.44.2'


In [3]:
from dataclasses_json import DataClassJsonMixin
from dataclasses import dataclass, field, fields
from typing import Optional
import random
from src.dataset import BridgeSample, BridgeRelation, BridgeDataset
from src.dataset import load_bridge_relation, load_bridge_relations, load_bridge_dataset        

In [4]:
relations = load_bridge_relations()
dataset = BridgeDataset(relations)

2024-08-30 22:16:34 src.dataset INFO     initialized bridge relation superpower_characters with 27 examples
2024-08-30 22:16:34 src.dataset INFO     initialized bridge relation sport_players with 23 examples
2024-08-30 22:16:34 src.dataset INFO     initialized bridge relation movie_actor with 52 examples
2024-08-30 22:16:34 src.dataset INFO     initialized bridge relation architect_building with 21 examples
2024-08-30 22:16:34 src.dataset INFO     initialized bridge dataset with 4 relations and 119 examples


In [6]:
len(dataset.relations)

4

In [10]:
import json


known_data_file = "filtered_2024-07-30T17:30:08.336365.json"
cached_known_dir = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "bridge_cached", "Meta-Llama-3-8B-Instruct"
)
with open(os.path.join(cached_known_dir, known_data_file), "r") as f:
    json_data = json.load(f)
# dataset = BridgeDataset.from_dict(json_data)

In [15]:
from src.dataset import BridgeRelation
relation = BridgeRelation.from_dict(json_data["relations"][0])
dataset = BridgeDataset([relation])

2024-08-30 22:20:24 src.dataset INFO     initialized bridge relation superpower_characters with 26 examples
2024-08-30 22:20:24 src.dataset INFO     initialized bridge dataset with 1 relations and 25 examples


In [16]:
q,a = dataset[10]

print(q)
print(a)

Given two entities, find a common link between them.
#
What is a common link between Ra's al Ghul and Vandal Savage?
A: immortal - an attribute that both characters Ra's al Ghul and Vandal Savage possess.
#
What is a common link between Venom and Carnage?
A:
symbiote - an attribute that both characters Venom and Carnage possess.


In [17]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

# model_key = "meta-llama/Meta-Llama-3-8B"
model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2024-08-30 22:21:01 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]

2024-08-30 22:21:07 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-8B-Instruct> | size: 15316.516 MB | dtype: torch.float16 | device: cuda:0


In [18]:
sample_idx = 17

prompt, answer = dataset[sample_idx]
answer

'fire control - an attribute that both characters Human Torch and Firestorm possess.'

In [19]:
from src.models import prepare_input

# inputs = prepare_input(
#     prompts = prompt,
#     tokenizer=mt,
#     add_bos_token=False
# )

# generation = mt._model.generate(
#     **inputs,
#     max_new_tokens=100,
#     top_k = 1
# )

# print(mt.tokenizer.decode(generation[0], skip_special_tokens=False))

In [20]:
from src.functional import predict_bridge_entity

predicted_ans = predict_bridge_entity(mt, prompt)
predicted_ans

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


' fire manipulation - an ability that both characters Human Torch and Firestorm possess.\n'

In [9]:
from src.functional import ask_gpt4o

query_sample = dataset.examples[sample_idx]
ask_gpt4o(query_sample, predicted_ans)

'Yes.'

In [14]:
from src.functional import filter_bridge_samples_by_model_knowledge

dataset = filter_bridge_samples_by_model_knowledge(mt, dataset)

  0%|          | 0/119 [00:00<?, ?it/s]

2024-07-30 17:16:03 src.functional INFO     ['Daredevil', 'Toph Beifong'] <> blind | predicted:  martial arts - a skill that both characters Daredevil and Toph Beifong possess.
 => (✗)


  1%|          | 1/119 [00:06<12:57,  6.59s/it]

2024-07-30 17:16:14 httpx INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-30 17:16:14 src.functional INFO     ['Human Torch', 'Firestorm'] <> fire control | predicted:  pyrokinesis - the ability to control fire, which is a superpower possessed by both Human Torch and Firestorm.
 => (✗)


  2%|▏         | 2/119 [00:17<18:05,  9.28s/it]

...

2024-07-30 17:30:01 httpx INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-30 17:30:01 src.functional INFO     ['Babe Ruth', 'Hank Aaron'] <> baseball | predicted:  Home runs - a record that both Babe Ruth and Hank Aaron hold in baseball.
 => (✗)


 99%|█████████▉| 118/119 [14:04<00:07,  7.46s/it]

2024-07-30 17:30:08 httpx INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-30 17:30:08 src.functional INFO     ['Susan Storm', 'Invisible Woman'] <> invisibility | predicted:  superhero - both Susan Storm and Invisible Woman are superheroes in the Marvel Comics universe.
 => (✗)


100%|██████████| 119/119 [14:11<00:00,  7.16s/it]

2024-07-30 17:30:08 src.functional INFO     filtered 95 samples out of 119 with 4 icl examples


In [15]:
import datetime

file_name = os.path.join(
    env_utils.DEFAULT_DATA_DIR, 
    "bridge_cached", 
    f"filtered_{datetime.datetime.now().isoformat()}.json"
)

with open(file_name, "w") as f:
    f.write(dataset.to_json())

In [16]:
# loaded = BridgeDataset.from_json(file_name)

In [18]:
import json
with open(file_name, "r") as f:
    json_data = json.load(f)
loaded_dataset = BridgeDataset.from_dict(json_data)

2024-07-30 17:53:57 src.dataset INFO     initialized bridge relation superpower_characters with 26 examples
2024-07-30 17:53:57 src.dataset INFO     initialized bridge relation sport_players with 22 examples
2024-07-30 17:53:57 src.dataset INFO     initialized bridge relation movie_actor with 51 examples
2024-07-30 17:53:57 src.dataset INFO     initialized bridge relation architect_building with 20 examples
2024-07-30 17:53:57 src.dataset INFO     initialized bridge dataset with 4 relations and 95 examples
